**Step-1: Load sales.csv as a DataFrame**

In [0]:
# Load the CSV as a DataFrame
sales_df= spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/shared_uploads/parveen.r@live.com/sales.csv")

In [0]:
# Peek at the data
sales_df.show(5)

In [0]:
display(sales_df)

Databricks visualization. Run in Databricks to view.

_**What happens?**_
- Driver Program sends the command to Spark.
- Spark splits the file into partitions (by default: based on file size, usually 128MB per partition).

**Step 2: Check the Number of Partitions**

In [0]:
print("Number of partitions:", sales_df.rdd.getNumPartitions())


_**What does this show?**_
- Spark split your file into this many chunks.
- Each partition will become a task sent to executors.

**Step 3: Transformation – Filter for Large Sales**

In [0]:
# Transformation: prepare a filter for big orders (e.g., > 1000 units)
large_sales = sales_df.filter(sales_df.Quantity > 4)

# No work happens yet—Spark just builds a plan!
print(large_sales)


_**Spark is planning what to do, but hasn’t done it yet.**_

_**Step 4: Action – Count the Results**_

In [0]:
# Action: actually count the large sales rows
count_large_sales = large_sales.count()
print("Number of large sales:", count_large_sales)


_Now Spark executes:_

- Driver asks for results.
- Spark’s Cluster Manager creates tasks for each partition.
- Each Executor gets a task (processes a chunk).
- Executors filter their chunk, return counts to Driver.
- Driver adds up results and shows you the final count.

**Step 5: See Tasks in the Spark UI**
- In Databricks, after you run the above cell, click on the “View Spark UI” link at the top of the cell.
- Go to the Stages tab:
  -   You’ll see as many tasks as there are partitions!
  -   Each task corresponds to a partition, run by an executor.
- You can see:
  - DAG visualization (the plan Spark builds)
  - Task distribution (how work is divided)

**Step 6: Repartition and See the Effect**

In [0]:
# Repartition to 8 partitions (forces more parallelism)
sales_df2 = sales_df.repartition(8)
print("Partitions after repartition:", sales_df2.rdd.getNumPartitions())

# Trigger an action
sales_df2.filter(sales_df2.Quantity > 4).count()
# Check Spark UI: you’ll see 8 tasks for the stage!


_You control parallelism and task distribution using repartitioning._

**Step 7: Aggregation Example (Group By Customer)**

In [0]:
# Calculate total sales amount per customer
from pyspark.sql.functions import col, sum as _sum

sales_df = sales_df.withColumn("TotalAmount", col("Quantity") * col("UnitPrice"))
customer_sales = sales_df.groupBy("CustomerID").agg(_sum("TotalAmount").alias("TotalSpent"))

customer_sales.orderBy("TotalSpent", ascending=False).show(5)


- Each partition processes a chunk.
- Spark’s Shuffle: after local aggregations, data is redistributed for the group by (“shuffle” stage).
- Tasks are assigned, results combined, and final result returned to Driver.